##Install ForecastFlowML

In [0]:
%pip install git+https://github.com/canerturkseven/forecastflowml

Python interpreter will be restarted.
Collecting git+https://github.com/canerturkseven/forecastflowml
 Cloning https://github.com/canerturkseven/forecastflowml to /tmp/pip-req-build-tux7xn5q
 Running command git clone -q https://github.com/canerturkseven/forecastflowml /tmp/pip-req-build-tux7xn5q
 Installing build dependencies: started
 Installing build dependencies: finished with status 'done'
 Getting requirements to build wheel: started
 Getting requirements to build wheel: finished with status 'done'
 Preparing wheel metadata: started
 Preparing wheel metadata: finished with status 'done'
Collecting pandas>=1.2.4
 Downloading pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
Collecting scikit-learn>=1.0.0
 Downloading scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
Requirement already satisfied: numpy>=1.17.0 in /databricks/python3/lib/python3.7/site-packages (from forecastflowml==0.0.1) (1.18.1)
Collecting pyspark>=2.3.2
 Downloading pyspark-3.3.2.tar.gz (281.4 MB)
Collecting plotly>=5.0.0
 Downloading plotly-5.13.1-py2.py3-none-any.whl (15.2 MB)
Requirement already satisfied: python-dateutil>=2.8.0 in /databricks/python3/lib/python3.7/site-packages (from forecastflowml==0.0.1) (2.8.1)
Collecting lightgbm>=3.0.0
 Downloading lightgbm-3.3.5-py3-none-manylinux1_x86_64.whl (2.0 MB)
Collecting pyarrow>=3.0.0
 Downloading pyarrow-11.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.1 MB)
Collecting optuna>=2.5.0
 Downloading optuna-3.1.0-py3-none-any.whl (365 kB)
Collecting mlflow>=1.30.0
 Downloading mlflow-1.30.0-py3-none-any.whl (17.0 MB)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.7/site-packages (from pandas>=1.2.4->forecastflowml==0.0.1) (2019.3)
Requirement already satisfied: scipy>=1.1.0 in /databricks/python3/lib/python3.7/site-packages (from scikit-learn>=1.0.0->forecastflowml==0.0.1) (1.4.1)
Requirement already satisfied: joblib>=0.11 in /databricks/python3/lib/python3.7/site-packages (from scikit-learn>=1.0.0->forecastflowml==0.0.1) (0.14.1)
Collecting threadpoolctl>=2.0.0
 Downloading threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
Collecting py4j==0.10.9.5
 Downloading py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
Collecting tenacity>=6.2.0
 Downloading tenacity-8.2.2-py3-none-any.whl (24 kB)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.7/site-packages (from python-dateutil>=2.8.0->forecastflowml==0.0.1) (1.14.0)
Requirement already satisfied: wheel in /databricks/python3/lib/python3.7/site-packages (from lightgbm>=3.0.0->forecastflowml==0.0.1) (0.34.2)
Collecting colorlog
 Downloading colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
Collecting sqlalchemy>=1.3.0
 Downloading SQLAlchemy-2.0.5.post1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.7 MB)
Collecting tqdm
 Downloading tqdm-4.65.0-py3-none-any.whl (77 kB)
Collecting PyYAML
 Downloading PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
Collecting cmaes>=0.9.1
 Downloading cmaes-0.9.1-py3-none-any.whl (21 kB)
Collecting packaging>=20.0
 Downloading packaging-23.0-py3-none-any.whl (42 kB)
Collecting alembic>=1.5.0
 Downloading alembic-1.10.2-py3-none-any.whl (212 kB)
Collecting click<9,>=7.0
 Downloading click-8.1.3-py3-none-any.whl (96 kB)
Collecting importlib-metadata!=4.7.0,<6,>=3.7.0
 Downloading importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
Collecting querystring-parser<2
 Downloading querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
Requirement already satisfied: protobuf<5,>=3.12.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow>=1.30.0->forecastflowml==0.0.1) (3.20.1)
Collecting sqlparse<1,>=0.4.0
 Downloading sqlparse-0.4.3-py3-none-any.whl (42 kB)
Collecting docker<7,>=4.0.0
 Downloading docker-6.0.1-py3-none-any.whl (147 kB)
Requirement already satisfied: requests<3,>=2.17.3 in /databricks/python3/lib/py

## Some Spark settings

In [0]:
# We have 2 cores in Databricks Spark cluster
spark.conf.set("spark.sql.shuffle.partitions", 2)
# Enable PyArrow for faster transformation to Pandas DF
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# Prevent combining small tasks. We want to run all models in parallel even though they are small.
spark.conf.set("spark.sql.adaptive.enabled", "false")

## Import packages

In [0]:
import mlflow
from forecastflowml.meta_model import MetaModel
from forecastflowml.preprocessing import FeatureExtractor
from forecastflowml.data.loader import load_walmart_m5
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

## Load sample dataset (Wallmart M5 Kaggle)

In [0]:
df = load_walmart_m5(spark)
df.show(n=10,truncate=False)
df.count()

+---------------------------+-----------+-------+------+--------+--------+-----+----------+---------+
id |item_id |dept_id|cat_id|store_id|state_id|sales|date |christmas|
+---------------------------+-----------+-------+------+--------+--------+-----+----------+---------+
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |2.0 |2011-01-29|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |5.0 |2011-01-30|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |3.0 |2011-01-31|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |0.0 |2011-02-01|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |0.0 |2011-02-02|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |0.0 |2011-02-03|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |0.0 |2011-02-04|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |1.0 |2011-02-05|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |0.0 |2011-02-06|0 |
FOODS_1_013_TX_2_evaluation|FOODS_1_013|FOODS_1|FOODS |TX_2 |TX |3.0 |2011-02-07|0 |
+---------------------------+-----------+-------+------+--------+--------+-----+----------+---------+
only showing top 10 rows

Out[3]: 1497336

# Feature Engineering

## Initialize feature extractor model

In [0]:
preprocessor = FeatureExtractor(
    id_col="id",
    date_col="date",
    date_frequency="day",
    target_col="sales",
    target_encodings=[
        {
            "partition_cols": ["item_id", "store_id"],
            "windows": [7, 14, 28],
            "lags": [7, 14, 21, 28],
            "functions": ["mean"],
        },
        {
            "partition_cols": ["item_id", "store_id"],
            "windows": [1],
            "lags": [7, 8, 9, 14, 15, 16, 21, 22, 23, 28, 29, 30],
            "functions": ["mean"],
        },
    ],
    date_features=[
        "day_of_month",
        "day_of_week",
        "week_of_year",
        "quarter",
        "month",
        "year",
    ],
    history_lengths=["item_id", ["item_id", "store_id"]],
    encode_events={"cols": ["christmas"], "window": 15},
    count_consecutive_values={"value": 0, "lags": [7, 14, 21, 28]},
)

## Create features and save dataframe as intermediate step

In [0]:
df_preprocessed = preprocessor.transform(df).localCheckpoint()
df_preprocessed.filter(F.col("date") >= "2014-01-01").show(n=5, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------
 date | 2014-01-01 
 id | FOODS_1_002_TX_1_evaluation 
 item_id_store_id_window_14_lag_7_mean | 0.14285714285714285 
 item_id | FOODS_1_002 
 item_id_store_id_window_1_lag_15_mean | 0.0 
 count_consecutive_value_lag_7 | 5 
 count_consecutive_value_lag_14 | 6 
 sales | 0.0 
 item_id_store_id_window_7_lag_21_mean | 0.0 
 item_id_store_id_window_14_lag_14_mean | 0.07142857142857142 
 item_id_store_id_window_1_lag_16_mean | 0.0 
 cat_id | FOODS 
 store_id | TX_1 
 item_id_store_id_window_1_lag_22_mean | 0.0 
 item_id_store_id_window_28_lag_7_mean | 0.10714285714285714 
 history_length_item_id_store_id | 1057 
 item_id_store_id_window_7_lag_28_mean | 0.14285714285714285 
 dept_id | FOODS_1 
 item_id_store_id_window_1_lag_30_mean | 0.0 
 item_id_store_id_window_1_lag_23_mean | 0.0 
 item_id_store_id_window_1_lag_29_mean | 0.0 
 item_id_store_id_window_1_lag_9_mean | 0.0 
 item_id_store_id_window_1_lag_28_mean | 0.0 
 item_id_store_id_window_7_lag_14_mean | 0.14285714285714285 
 item_id_store_id_window_1_lag_8_mean | 0.0 
 count_consecutive_value_lag_28 | 4 
 item_id_store_id_window_1_lag_7_mean | 0.0 
 item_id_store_id_window_1_lag_21_mean | 0.0 
 item_id_store_id_window_1_lag_14_mean | 0.0 
 item_id_store_id_window_7_lag_7_mean | 0.14285714285714285 
 item_id_store_id_window_14_lag_21_mean | 0.07142857142857142 
 item_id_store_id_window_28_lag_21_mean | 0.07142857142857142 
 item_id_store_id_window_28_lag_28_mean | 0.07142857142857142 
 item_id_store_id_window_28_lag_14_mean | 0.07142857142857142 
 history_length_item_id | 1057 
 count_consecutive_value_lag_21 | 11 
 state_id | TX 
 christmas | -7 
 item_id_store_id_window_14_lag_28_mean | 0.07142857142857142 
 day_of_month | 1 
 day_of_week | 4 
 week_of_year | 1 
 quarter | 1 
 month | 1 
 year | 2014 
-RECORD 1-------------------------------------------------------------
 date | 2014-01-02 
 id | FOODS_1_002_TX_1_evaluation 
 item_id_store_id_window_14_lag_7_mean | 0.07142857142857142 
 item_id | FOODS_1_002 
 item_id_store_id_window_1_lag_15_mean | 0.0 
 count_consecutive_value_lag_7 | 6 
 count_consecutive_value_lag_14 | 7 
 sales | 1.0 
 item_id_store_id_window_7_lag_21_mean | 0.14285714285714285 
 item_id_store_id_window_14_lag_14_mean | 0.07142857142857142 
 item_id_store_id_window_1_lag_16_mean | 0.0 
 cat_id | FOODS 
 store_id | TX_1 
 item_id_store_id_window_1_lag_22_mean | 0.0 
 item_id_store_id_window_28_lag_7_mean | 0.10714285714285714 
 history_length_item_id_store_id | 1058 
 item_id_store_id_window_7_lag_28_mean | 0.14285714285714285 
 dept_id | FOODS_1 
 item_id_store_id_window_1_lag_30_mean | 0.0 
 item_id_store_id_window_1_lag_23_mean | 0.0 
 item_id_store_id_window_1_lag_29_mean | 0.0 
 item_id_store_id_window_1_lag_9_mean | 0.0 
 item_id_store_id_window_1_lag_28_mean | 0.0 
 item_id_store_id_window_7_lag_14_mean | 0.0 
 item_id_store_id_window_1_lag_8_mean | 0.0 
 count_consecutive_value_lag_28 | 5 
 item_id_store_id_window_1_lag_7_mean | 0.0 
 item_id_store_id_window_1_lag_21_mean | 1.0 
 item_id_store_id_window_1_lag_14_mean | 0.0 
 item_id_store_id_window_7_lag_7_mean | 0.14285714285714285 
 item_id_store_id_window_14_lag_21_mean | 0.14285714285714285 
 item_id_store_id_window_28_lag_21_mean | 0.10714285714285714 
 item_id_store_id_window_28_lag_28_mean | 0.07142857142857142 
 item_id_store_id_window_28_lag_14_mean | 0.07142857142857142 
 history_length_item_id | 1058 
 count_consecutive_value_lag_21 | 0 
 state_id | TX 
 christmas | -8 
 item_id_store_id_window_14_lag_28_mean | 0.07142857142857142 
 day_of_month | 2 
 day_of_week | 5 
 week_of_year | 1 
 quarter | 1 
 month | 1 
 year | 2014 
-RECORD 2-------------------------------------------------------------
 date | 2014-01-03 
 id | FOODS_1_002_TX_1_evaluation 
 item_id_store_id_window_14_lag_7_mean | 0.07142857142857142 
 item_id | FOODS_1_002 
 item_id_store_id_window_1_lag_15_mean | 0.0 
 count_consecutive_value_lag_7 | 7 
 coun

#Training

##Split dataset into train and test

In [0]:
df_train = df_preprocessed.filter(F.col("date") <= "2016-05-22")
df_test = df_preprocessed.filter(F.col("date") > "2016-05-22")

## Initialize meta model

In [0]:
model = MetaModel(
    
    # dataset parameters
    group_col="state_id",  # column to slice dataframe
    id_cols=["id"],  # columns to use as time series identifier
    date_col="date",  # date column
    target_col="sales",  # target column
    date_frequency="days",  # date frequency (days, weeks, months, years) of dataset
    
    # model parameters
    model_horizon=7,  # horizon per model
    max_forecast_horizon=28,  # total forecast horizon
    lag_feature_range=2,  # extra lags to include as features based on model horizon
    
    # cross validation and optimisation parameters
    n_cv_splits=1,  # number of time-based cv splits
    cv_step_length=28,  # number of dates between each cv folds
    max_hyperparam_evals=1,  # total number of optuna trials
    scoring="neg_mean_squared_error",  # sklearn scoring metric
    
    # optuna hyperparameter space
    hyperparam_space_fn=lambda trial: {
        "objective": trial.suggest_categorical("objective", ['tweedie']),
        "num_leaves": trial.suggest_int("num_leaves", 50, 50),
        "n_estimators": trial.suggest_int("n_estimators", 30, 30)
    },
    
    # mlflow parameters
    tracking_uri="databricks",  # Mlflow tracking URI. Provide "./mlruns" in case of running locally.
)

## Train the model

In [0]:
model.train(df_train)

2023/03/11 06:28:09 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-10-19; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'forecastflowml'}
/databricks/spark/python/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
 "more details.", UserWarning)

In [0]:
model.cv_forecast_graph['TX']

In [0]:
model.cv_forecast_graph["CA"]

In [0]:
model.feature_importance_graphs['TX']['horizon_1']

## All artifacts/metrics can be found in MLFlow UI 
<img src="/files/tables/1-2.PNG" width=50% height=50%>

## Models and feature schemas
<img src="/files/tables/3.PNG" width=50% height=50%>

## Forecast on time based cross validation folds
<img src="/files/tables/4.PNG" width=70% height=70%>

## Feature importance
<img src="/files/tables/5.PNG" width=70% height=70%>

## Bayesian optimisation visualisations
<img src="/files/tables/6.PNG" width=70% height=70%>

# Prediction

## Load meta model

In [0]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{model.run_id}/meta_model")

## Make predictions

In [0]:
loaded_model.predict(df_test).show(truncate=False)